In [2]:
# import relevant libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
plt.rc('figure',figsize =(20,10))
sns.set(style='whitegrid')

In [7]:
# import the energy data
energy_data = pd.read_csv("C:\\Users\\D\\Downloads\\energydata_complete.csv")

In [8]:
#Overview of the data set
energy_data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [10]:
# droped the columns date and lights
energy_data.drop(['date','lights'], axis=1, inplace = True)

In [12]:
# normalised the data 
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(energy_data), columns= energy_data.columns)

C:\Users\D\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [26]:
# creating a simple linear regression model using colums T2(as independent_variable) and T6 as dependent_variable
Independent_variable = normalised_df[['T2']]
dependent_variable = normalised_df[['T6']]
x_train, x_test, y_train, y_test = train_test_split(Independent_variable, dependent_variable, test_size=0.3, 
                                                    random_state=42)
linear_model = LinearRegression()
#fit the model to the training dataset
linear_model.fit(x_train, y_train)
#obtain predictions
predicted_values = linear_model.predict(x_test)

In [27]:
#mean absolute error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 3) 

0.082

In [28]:
#r2_score = r2_score(y_test, predicted_values)
r2_score = linear_model.score(x_test, y_test)
round(r2_score, 3)


0.643

In [29]:
# creating a multiple linear regeression model using the normalised data with the Appliances as target variable
features_df = normalised_df.drop(columns=['Appliances'])
energy_target = normalised_df['Appliances']
x_train, x_test, y_train, y_test = train_test_split(features_df,energy_target, test_size=0.3, 
                                                    random_state=42)
linear_model = LinearRegression()
#fit the model to the training dataset
linear_model.fit(x_train, y_train)
#obtain predictions
predicted_values = linear_model.predict(x_test)

In [30]:
# calculating the mean absolute error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 3) 

0.05

In [40]:
# calculating the R squared
r2_score = linear_model.score(x_test, y_test)
round(r2_score,2 )

0.15

In [38]:
# calculating the RSS
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2)

45.35

In [34]:
#calculating the root mean squared error
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.088

In [60]:
#creating a Ridge regression model
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)
predict_values = ridge_reg.predict(x_test)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predict_values)
print( "Mean Absolute Error: {}".format(round(mae, 3)))
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
print('Root Mean Squared Error:{}'.format(round(rmse, 3)))

Mean Absolute Error: 0.05
Root Mean Squared Error:0.088


In [61]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)
predicts_values = lasso_reg.predict(x_test)
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
print('Root Mean Squared Error:{}'.format(round(rmse, 3)))

Root Mean Squared Error:0.088


In [62]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
print(final_weights)
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
print(final_weights)

       Features  Linear_Model_Weight  Ridge_Weight
0          RH_2            -0.456698     -0.411071
1         T_out            -0.321860     -0.262172
2            T2            -0.236178     -0.201397
3            T9            -0.189941     -0.188916
4          RH_8            -0.157595     -0.156830
5        RH_out            -0.077671     -0.054724
6          RH_7            -0.044614     -0.045977
7          RH_9            -0.039800     -0.041367
8            T5            -0.015657     -0.019853
9            T1            -0.003281     -0.018406
10          rv2             0.000770      0.000748
11          rv1             0.000770      0.000748
12  Press_mm_hg             0.006839      0.006584
13           T7             0.010319      0.010098
14   Visibility             0.012307      0.012076
15         RH_5             0.016006      0.016152
16         RH_4             0.026386      0.024579
17           T4             0.028981      0.027384
18    Windspeed             0.0